In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

from src.utils.labels import Labels
import src.utils.las_utils as las_utils
import src.scrapers.ams_bgt_scraper as ams_bgt_scraper
from src.utils.csv_utils import write_csv
from src.pipeline import Pipeline
from src.fusion.car_fuser import CarFuser

## Car fuser based on cluster dimensions and BGT information


In [ ]:
# The already labelled ground and building points must be removed.
exclude_labels = (Labels.GROUND, Labels.BUILDING)

In [ ]:
# Select the file to process.
in_file = '../datasets/pointcloud/grown_2397_9705.las'
out_file = '../datasets/pointcloud/cars_2397_9705.las'

# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'
pc_data_folder = '../datasets/pointcloud/'

In [ ]:
# Specify the bounding box of the area we want to scrape.
bbox = las_utils.get_bbox_from_las_folder(pc_data_folder)

# Road parking spots layers in BGT
bgt_layers = ["BGT_WGL_rijbaan_lokale_weg", "BGT_WGL_parkeervlak",
             "BGT_WGL_rijbaan_autoweg", "BGT_WGL_rijbaan_autosnelweg",
             "BGT_WGL_rijbaan_regionale_weg"]

In [ ]:
# Scrape data from the Amsterdam WFS and parse the json.
bgt_road_polygons_csv = []
for layer in bgt_layers:
    # Scrape data from the Amsterdam WFS, this will return a json response.
    json_content = ams_bgt_scraper.scrape_amsterdam_bgt(layer, bbox=bbox)
    
    # Parse the downloaded json response.
    parsed_content, csv_headers = ams_bgt_scraper.parse_polygons(json_content,
                                                                 offset_meter=-0.3, 
                                                                 prepare_csv=True)
    bgt_road_polygons_csv += parsed_content

# Write the csv
bgt_data_file = '../datasets/bgt/bgt_roads.csv'
write_csv(bgt_data_file, bgt_road_polygons_csv, csv_headers)

In [ ]:
# Car fuser using a clustering algorithm and BGT road data.
car_fuser = CarFuser(Labels.CAR, exclude_labels, ahn_data_folder, 
                     bgt_file=bgt_data_file, octree_level=10,
                     min_component_size=100, max_above_ground=3,
                     min_width_thresh=1.5, max_width_thresh=2.55,
                     min_length_thresh=2.0, max_length_thresh=7.0)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (car_fuser,)
pipeline = Pipeline(process_sequence=process_sequence)

## Process a single file

In [ ]:
# Process the file.
start = time.time()
pipeline.process_file(in_file, out_file=out_file)
end = time.time()
print(f'Tile labelled in {end-start:.2f} seconds.')